In [1]:
import scipy
import math
import numpy as np
import sys

import pymunk
from pymunk.pygame_util import *
from pymunk.vec2d import Vec2d
import pygame
from pygame.locals import *

pygame 2.1.0 (SDL 2.0.16, Python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Variable/constant
size = 800
FPS = 50
stiffness = 5000.0
damping = 100

In [ ]:
#Pygame settings
space = pymunk.Space()
space.gravity = 0,-450
space.damp = 0.999

GRAY = (220, 220, 220)
RED = (255, 0, 0)
BLACK = (0,0,0)
WHITE = (255,255,255)

#functions
def convert_position(point):
    return int(point[0]), int(size-point[1])

class Segment():    
    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.body = pymunk.Body(body_type=pymunk.Body.STATIC)
        self.shape = pymunk.Segment(self.body,(x,y),(size,y),5)
        self.shape.density = 0.01
        self.shape.elasticity = 0.05
        space.add(self.body,self.shape)
        
    def draw(self,screen):
        pygame.draw.line(screen,(0,0,0),(self.x,size-self.y),(size,size-self.y),5)




#Object class
class Particle():
    def __init__(self,x,y):
        self.body = pymunk.Body()
        self.body.position = x, y
        self.shape = pymunk.Circle(self.body,3)
        self.shape.density = 1
        self.shape.elasticity = 1
        space.add(self.body,self.shape)
        
    def draw(self,screen):
        pygame.draw.circle(screen, (255,0,0), convert_position(self.body.position), 5)
            
        
class DampedSpring():
    def __init__(self, body1, attachment, identifier="body"):
        self.body1 = body1
        if  identifier == "body":
            self.body2 = attachment
        elif identifier == "position":
            self.body2 = pymunk.Body(body_type=pymunk.Body.STATIC)
            self.body2.position = attachment
        rl = self.body1.position.get_distance(self.body2.position) * 0.9
        joint = pymunk.DampedSpring(self.body1, self.body2, (0, 0), (0, 0), rl, stiffness, damping)
        joint.max_bias = 100
        space.add(joint)
    
    def draw(self,screen):
        pos1 = convert_position(self.body1.position)
        pos2 = convert_position(self.body2.position)
        pygame.draw.line(screen, (0,0,0), pos1, pos2, 5)
        
#function to create square            
class Square():
   
    def __init__(self):
        
        # TODO MOVE FLOOR INTO ITS OWN CLASS SO WE CAN REFRACTOR
        # SEGMENTS = STATICS OBJECT
        self.Segments = []
        self.Segments.append(Segment(0,50))
        
        # PARTICLES MAKING SQUARE
        # we can make it auto x,y here #optional fix later
        self.Particles = []
        self.Particles.append(Particle(350,600)) #0
        self.Particles.append(Particle(450,600)) #1
        self.Particles.append(Particle(350,500)) #2
        self.Particles.append(Particle(450,500)) #3
        
        # CONSTRAINTS = DAMPED SPRING CONSTRAINTS CONNECTING THE PARTICLES
        self.Springs = []
        self.Springs.append(DampedSpring(self.Particles[0].body,self.Particles[1].body))
        self.Springs.append(DampedSpring(self.Particles[0].body,self.Particles[2].body))
        self.Springs.append(DampedSpring(self.Particles[0].body,self.Particles[3].body))
        self.Springs.append(DampedSpring(self.Particles[2].body,self.Particles[1].body))
        self.Springs.append(DampedSpring(self.Particles[2].body,self.Particles[3].body))
        self.Springs.append(DampedSpring(self.Particles[1].body,self.Particles[3].body))
        

        
def distance(event,particle):
#     print("------XY------")
#     print(abs(event[0] - particle[0]))
#     print(abs(event[1] - particle[1]))
    return abs(event[0] - particle[0]) < 10 and abs(event[1] - particle[1]) < 10 
        
        
#Simluation loop
class Simulation():
    
    def __init__(self):
        pygame.init()
        self.running = True
        self.screen = pygame.display.set_mode((size,size))
        self.clock = pygame.time.Clock()
        self.pulling = False
        self.active_shape = None
        self.selected_shapes = []
        
        #SETUP
        self.square = Square() #TODO MAKE A SELF.SHAPES = [] (a list) so easier to self.draw() later on
        pygame.display.set_caption('Soft-Body Simulation (Square)')
    
    def do_event(self, event):
        
        if event.type == QUIT:
            self.running = False 
            
        elif event.type == MOUSEMOTION:
            self.p = event.pos
            
        # P: Position of the mouse when click    
        elif event.type == MOUSEBUTTONDOWN:
            p = from_pygame(event.pos, self.screen)
            self.active_shape = None
            fixed_pos = (event.pos[0],size-event.pos[1])
            for s in space.shapes:
                query = s.point_query(p)
                point = query.point
                if distance(fixed_pos,point):
                    self.active_shape = s
                    self.pulling = True
                    s.body.angle = (p - s.body.position).angle
                    if pygame.key.get_mods() and KMOD_META:
                        self.selected_shapes.append(s)
                    else:
                        self.selected_shapes = []         
        elif event.type == MOUSEBUTTONUP:
            if self.pulling:
                self.pulling = False
                b = self.active_shape.body
                p0 = Vec2d(b.position.x,b.position.y)
                p1 = from_pygame(event.pos, self.screen)
                impulse = 100 * (p0 - p1).rotated(b.angle)
                b.apply_impulse_at_local_point(impulse)        
        
        
        
    def run(self):
        while self.running:
            for event in pygame.event.get():
                self.do_event(event)
            self.draw()
            self.clock.tick(FPS)
            space.step(1/FPS)
        pygame.quit()
        
        
    # you can have better draw function bro this is too dumb
    def draw(self):
        self.screen.fill(WHITE)
        for particle in self.square.Particles:
            particle.draw(self.screen)
        for spring in self.square.Springs:
            spring.draw(self.screen)
        for segment in self.square.Segments:
            segment.draw(self.screen)
        pygame.display.update()

            
            
        
if __name__ == '__main__':
    Simulation().run()